In [68]:
import torch
import torchvision
from torchvision import datasets,transforms, models
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable 
import time
%matplotlib inline

In [74]:
path = "./data"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
                             # transforms.Lambda(shear),
                             transforms.RandomSizedCrop(224),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),
                             normalize,
                                 ])

data_image = {x:datasets.ImageFolder(root = os.path.join(path,x),
                                     transform = transform)
              for x in ["train", "val"]}

data_loader_image = {x:torch.utils.data.DataLoader(dataset=data_image[x],
                                                num_workers=4,
                                                batch_size=8,
                                                pin_memory=True,
                                                shuffle = True)
                     for x in ["train", "val"]}

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:397: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.


In [75]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [76]:
classes = data_image["train"].classes
classes_index = data_image["train"].class_to_idx
print(classes)
print(classes_index)

['cat', 'dog']
{'cat': 0, 'dog': 1}


In [77]:
print(u"训练集个数:", len(data_image["train"]))
print(u"验证集个数:", len(data_image["val"]))

训练集个数: 22500
验证集个数: 2500


In [ ]:
X_train, y_train = next(iter(data_loader_image["train"]))
mean = [0.5,0.5,0.5]
std  = [0.5,0.5,0.5]
img = torchvision.utils.make_grid(X_train)
img = img.numpy().transpose((1,2,0))
img = img*std+mean

print([classes[i] for i in y_train])
plt.imshow(img)

In [79]:
model = models.vgg16(pretrained=True)

In [ ]:
print(model)

In [80]:
lr = 1e-2
for parma in model.parameters():
    parma.requires_grad = False

model.classifier = torch.nn.Sequential(torch.nn.Linear(25088, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 2))

for index, parma in enumerate(model.classifier.parameters()):
    if index == 6:
        parma.requires_grad = True
    
if use_gpu:
    model = model.cuda()


cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(),lr, weight_decay=1e-4)

In [ ]:
print(model)

In [85]:
n_epochs = 2
for epoch in range(n_epochs):
    since = time.time()
    print("Epoch{}/{}".format(epoch, n_epochs))
    print("-"*10)
    for param in ["train", "val"]:
        if param == "train":
            model.train = True
            model = torch.nn.DataParallel(model)
        else:
            model.train = False

        running_loss = 0.0
        running_correct = 0 
        batch = 0
        for data in data_loader_image[param]:
            batch += 1
            X, y = data
            if use_gpu:
                X, y  = Variable(X.cuda()), Variable(y.cuda())
            else:
                X, y = Variable(X), Variable(y)
        
            optimizer.zero_grad()
            y_pred = model(X)
            _, pred = torch.max(y_pred.data, 1)
        
            loss = cost(y_pred, y)
            if param =="train":
                loss.backward()
                optimizer.step()
            running_loss += loss.data[0]
            running_correct += torch.sum(pred == y.data)
            if batch%500 == 0 and param =="train":
                print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}".format(
                      batch, running_loss/(4*batch), 100*running_correct/(4*batch)))
            
        epoch_loss = running_loss/len(data_image[param])
        epoch_correct = 100*running_correct/len(data_image[param])

        print("{}  Loss: {:.4f},  Correct: {:.4f}".format(param, epoch_loss, epoch_correct))
    now_time = time.time() - since   
    print("Training time is:{:.0f}m {:.0f}s".format(now_time//60, now_time%60))

Epoch0/2
----------
Batch 500, Train Loss:151.1232, Train ACC:182.2000
Batch 1000, Train Loss:145.9629, Train ACC:182.3750
Batch 1500, Train Loss:151.4611, Train ACC:182.4667
Batch 2000, Train Loss:141.7430, Train ACC:183.0375
Batch 2500, Train Loss:143.9511, Train ACC:182.6500
train  Loss: 70.3320,  Correct: 91.4800
val  Loss: 70.3288,  Correct: 92.2000
Training time is:2m 4s
Epoch1/2
----------
Batch 500, Train Loss:126.0623, Train ACC:183.6000
Batch 1000, Train Loss:154.0388, Train ACC:184.0000
Batch 1500, Train Loss:145.3600, Train ACC:184.3500
Batch 2000, Train Loss:151.4245, Train ACC:184.0250
Batch 2500, Train Loss:168.2391, Train ACC:184.1700
train  Loss: 83.3328,  Correct: 92.0178
val  Loss: 110.4827,  Correct: 92.6000
Training time is:2m 5s


In [86]:
torch.save(model.state_dict(), "model_vgg16_finetune2.pkl")

In [ ]:
model.load_state_dict(torch.load('model_vgg16_finetune2.pkl'))

In [ ]:
dir(torch)

In [87]:
class ImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, path

In [108]:
data_test_img = ImageFolder(root="./test/",
                                     transform = transform)
data_loader_test_img = torch.utils.data.DataLoader(dataset=data_test_img,
                                                   batch_size = 1)

In [109]:
for a,b,c in data_loader_test_img:
    print(a)
    print(b)
    print(c)
    break


( 0 , 0 ,.,.) = 
 -2.0323 -2.0152 -1.9467  ...  -1.6384 -1.6213 -1.6042
 -2.0494 -2.0665 -2.0494  ...  -1.6555 -1.6727 -1.6727
 -2.0152 -1.9980 -1.9809  ...  -1.6898 -1.7069 -1.7069
           ...             ⋱             ...          
 -0.6794 -0.7308 -0.5596  ...   0.7591  0.8276  0.8618
 -0.5424 -0.6623 -0.5596  ...   0.7419  0.7933  0.8276
 -0.5082 -0.4911 -0.1314  ...   0.7248  0.7591  0.7933

( 0 , 1 ,.,.) = 
 -1.7381 -1.7206 -1.6681  ...  -1.2129 -1.1954 -1.1779
 -1.8256 -1.8606 -1.8782  ...  -1.1954 -1.1954 -1.1954
 -1.8256 -1.8431 -1.8606  ...  -1.2129 -1.2129 -1.2129
           ...             ⋱             ...          
 -0.8277 -0.9328 -0.7927  ...   0.9405  1.0105  1.0455
 -0.5126 -0.6877 -0.6176  ...   0.9230  0.9755  1.0105
 -0.4076 -0.4076 -0.0924  ...   0.9055  0.9405  0.9755

( 0 , 2 ,.,.) = 
 -1.0201 -0.9853 -0.8981  ...  -0.2010 -0.1661 -0.1487
 -1.2467 -1.2641 -1.2467  ...  -0.1661 -0.1487 -0.1487
 -1.3339 -1.3164 -1.3339  ...  -0.2184 -0.2010 -0.1835
           

In [110]:
import collections
clip = 0.001
cnt = 1
csv_map = {}

for image, label, path in (data_loader_test_img):
    images = Variable(image.cuda())
    y_pred = model(images)
    smax = torch.nn.Softmax()
    smax_out = smax(y_pred)[0]
    cat_prob = smax_out.data[0]
    dog_prob = smax_out.data[1]
    prob = dog_prob

    if cat_prob > dog_prob:
        prob = 1 - cat_prob
    prob = np.around(prob, decimals=4)
    prob = np.clip(prob, clip, 1-clip)
    filepath = path[0].split('/')[-1].split('.')[-2]
    if filepath == '0':
        print(path)
        break
    csv_map[filepath] = prob
    


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


In [111]:
csv_list = []
for k,v in csv_map.items():
    csv_list.append((k,v))

In [112]:
len(csv_list)

12500

In [ ]:
import pandas as pd

In [113]:
df = pd.DataFrame(csv_list)
df.columns = ['id','label']
df.id = df.id.astype(int)
df = df.sort_values('id')

In [59]:
df.to_csv('result3.csv',index=False)

In [ ]:
df.groupby('id').count()

In [ ]:
img = torchvision.utils.make_grid(image)
img = img.numpy().transpose(1,2,0)
mean = [0.5,0.5,0.5]
std  = [0.5,0.5,0.5]
img = img*std+mean
print("Pred Label:",[classes[i] for i in pred])
plt.imshow(img)